<img width="20%" alt="EarthDaily Agro" src="https://earthdaily.com/hubfs/EDA_logo_horizontal_gradient-navy_1.svg" style="border-radius: 12px" />

# Spatial Unit Initialization

> Onboard new areas of interest and monitor ingestion jobs end-to-end.

**Notebook map**
- ✅ Setup & authentication (load `.env`, create an API session)
- 🧱 Submit a multi-geometry payload (point, buffered point, polygon)
- ⏱️ Poll the ingestion job until a summary is available

> 💡 _Prereqs_: ensure `.env` contains the API base URL + client credentials and that the `lib/` package is importable from this repo.


## Before you run

- `ApiSession.from_env()` reuses the credentials injected via `.env`; switch tenants by editing that file once.
- Edit the geometry payload in the next section to demo different AOIs. The job is idempotent thanks to `options.idempotency_key`.
- The notebook purposely stays minimal so that it can run inside workshops without hidden state.


In [4]:
# --- bootstrap, environment, and API session ------------------------------------
import os
import sys
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, Markdown


# Project root = parent of this notebook's folder (1-init_db/)
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))


# Keep auth material in notebooks_v2/.env so repo roots stay clean
env_path = Path('..') / '.env'
if env_path.exists():
    load_dotenv(env_path)

DISCOVER_TOKEN = os.getenv("DISCOVER_TOKEN")
if not DISCOVER_TOKEN:
    raise RuntimeError("Missing DISCOVER_TOKEN in .env (required for polygon/bbox discovery).")

# Import helpers from lib
from lib import ApiSession, items_to_geoms, preview_geometries_map, wait_for_job
from lib.analytics import apply_notebook_style
apply_notebook_style()

session = ApiSession.from_env()
session

ApiSession(base_url='https://yiokf3wlwxeppc6yui6kdev25e0vtqtr.lambda-url.us-east-1.on.aws', token_manager=IdentityTokenManager(username='Bundles_ServiceUser', password='tAW32yUeDNQeBfzp', refresh_margin_seconds=300, validity_window_seconds=3600), timeout=60)

## Quick navigation
- [Before you run](#Before-you-run)
- [1. Trigger spatial unit ingestion](#1.-Trigger-spatial-unit-ingestion)
- [2. Poll the job until completion](#2.-Poll-the-job-until-completion)

## 1. Trigger spatial unit ingestion

Provide one or more geometries (raw WKT, point + optional radius, or a bbox). Each entry in `payload["items"]` becomes a spatial unit.

| Field | Notes |
| --- | --- |
| `type` | `POINT`, `POINT_RADIUS`, `BBOX`, or `WKT` |
| `lon`/`lat`/`radius_m` | Required for point-based payloads |
| `value` | Use WKT for polygons; long strings are fine |
| `options.idempotency_key` | Prevents duplicate jobs if the cell is re-run |
| `options.srid` | Defaults to `4326`; adjust for local projections if needed |

> ✏️ Update the collection below to match the shapes you want to ingest.


In [ ]:
# Customize the spatial units you want to ingest (mix and match geometry types).
payload = {
    "items": [
        {"type": "POINT", "lon": -94.7203, "lat": 44.2505, "radius_m": 500},
        {
            "label": "AOI polygon",
            "type": "wkt",
            "value": "POLYGON ((-94.65769669000616 44.25048284611244, -94.65800279739844 44.24607247866539, -94.6589087256991 44.24170490686098, -94.66040562153817 44.237422178804046, -94.66247895039672 44.23326551684501, -94.6651086411286 44.22927492146055, -94.66826928323373 44.22548878719971, -94.67193037483094 44.221943534348874, -94.67605661881908 44.21867325980829, -94.68060826428255 44.21570941047953, -94.68554148979818 44.21308048224219, -94.69080882493432 44.21081174734897, -94.69635960590416 44.208925012794204, -94.70214046104353 44.207438411916186, -94.7080958215316 44.20636623117867, -94.71416845256185 44.20571877374519, -94.7203 44.205502261116536, -94.72643154743815 44.20571877374519, -94.7325041784684 44.20636623117867, -94.73845953895648 44.207438411916186, -94.74424039409584 44.208925012794204, -94.74979117506568 44.21081174734897, -94.75505851020182 44.21308048224219, -94.75999173571745 44.21570941047953, -94.76454338118093 44.21867325980829, -94.76866962516907 44.221943534348874, -94.77233071676628 44.22548878719971, -94.7754913588714 44.22927492146055, -94.7781210496033 44.23326551684501, -94.78019437846183 44.237422178804046, -94.7816912743009 44.24170490686099, -94.78259720260156 44.24607247866539, -94.78290330999384 44.25048284611244, -94.78260651446156 44.25489353974701, -94.78170954017224 44.259262077578775, -94.78022089639904 44.26354637437918, -94.77815480053916 44.267705147512046, -94.7755310457788 44.271698315369655, -94.77237481450182 44.27548738454453, -94.76871643908322 44.279035821963326, -94.76459111224214 44.28230940834478, -94.76003854964782 44.28527656951522, -94.75510260796733 44.287908682322715, -94.74983086201223 44.29018035213324, -94.7442741450741 44.29206965916505, -94.73848605693287 44.29355837122074, -94.73252244436974 44.29463212070502, -94.72644085931437 44.29528054416805, -94.7203 44.29549738298498, -94.71415914068564 44.29528054416805, -94.70807755563027 44.29463212070502, -94.70211394306713 44.29355837122074, -94.6963258549259 44.29206965916506, -94.69076913798777 44.29018035213324, -94.68549739203267 44.28790868232273, -94.68056145035219 44.285276569515226, -94.67600888775786 44.28230940834478, -94.67188356091678 44.279035821963326, -94.66822518549819 44.27548738454453, -94.66506895422121 44.271698315369655, -94.66244519946083 44.267705147512046, -94.66037910360096 44.26354637437918, -94.65889045982776 44.25926207757878, -94.65799348553844 44.25489353974701, -94.65769669000616 44.25048284611244))",
        },
    ],
    "options": {"srid": 4326, "idempotency_key": "new_test", "discover_token": DISCOVER_TOKEN},
}

### Visualize geometries before ingestion
Use folium to preview the AOIs you are about to send. Adjust simplification/clustering if the map lags.


In [ ]:
geoms = items_to_geoms(payload['items'])
preview_geometries_map(
    geometries=geoms,
    simplify_tolerance=0.0005,
    max_features=300,
    render_mode='geometry',
    cluster_points=True,
    add_layer_control=True,
)

In [ ]:
# Trigger ingestion
response = session.post('/v1/spatial-units', json=payload)
response

## 2. Poll the job until completion

Use `wait_for_job` to watch the ingestion job and render the summary when it flips to a terminal status. The helper accepts any callable that returns the latest job payload so you can swap in a custom logger if desired.


In [ ]:
job_id = response['job_id']


def fetch_status(session: ApiSession, job_id: str):
    return session.get(f'/v1/jobs/{job_id}')


res = wait_for_job(session, job_id, fetch_status)
display(Markdown(res["summary"]))